## This script will calculate the nucleation rate using ice fractions

In [4]:
%matplotlib notebook
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import h5py
import pandas as pd
import os
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
from scipy.interpolate import RegularGridInterpolator

In [5]:
def read_hdf5(path, filename):
    f = h5py.File('{0}/{1}'.format(path, filename), 'r')
    return f['data']['time'], f['data']['Tave']

def T_at_distance(distance, velocity, droplet_size, T_function):
    '''Distance in m, velocity in m/s, droplet_size is diameter in micrometer'''
    time = np.divide(distance, velocity)
    temperature = T_function([droplet_size, time], method='linear')
    return temperature

def T_at_time(time, droplet_size, T_function):
    '''Time in s, droplet_size is diameter in micrometer'''
    temperature = T_function([droplet_size, time], method='linear')
    return temperature


#These are the sizes that are in the folder of hdf5 files. Must have the same timesteps
sizes = [3, 5, 10, 12, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

#sizes is the x 
x = sizes

# y is the timesteps, and this is taken from the file t_calib_22.h5
time, _ = read_hdf5('/Volumes/harddrive/codes_rays/temperature_calib/', 't_calib_18.h5')
y = list(time[:])  

#Create an empy matrix with the size of the sizes and times
temperatures = np.zeros([len(x), len(y)])

#Filling the matrix with actual values measured.
for size_index, size_value in enumerate(sizes):
    _, temperatures[size_index] = read_hdf5(
        '/Volumes/harddrive/codes_rays/temperature_calib/', 't_calib_{0}.h5'.format(size_value))

# This is a function returned by the RegularGridInterpolator
#interpolate_T = interp2d(xx, yy, temperatures, kind='linear',bounds_error=True)
interpolate_T = RegularGridInterpolator((x, y), temperatures)


In [6]:
diameter = 25.7*10E-6 #m
Volume = (4*np.pi/3)*(diameter/2)**3
flow_rate = (68.5*10E-6)/60*10E-3 #l/s
actual_frequency = flow_rate/Volume
spacing = 95.7*10E-6 #m
velocity = spacing * actual_frequency #m/s
distance = np.array([10,20,30,35,40,45,50,55,60])#mm
time = distance*10**-3/velocity #s
#temperature = T_at_time(time, 25.7, interpolate_T)
temperatures = [T_at_time(x, 25.7, interpolate_T) for x in time]
fraction = np.array([0,0,0,0.917431193,1.282051282,1.25,0,2.352941176,25])

In [10]:
# calculate J (nucleation)
J = np.zeros(len(distance)-1)
for k in range(len(fraction)):
    if k!=0:
        J[k-1]= -np.log((1-fraction[k])/(1-fraction[k-1]))/((time[k]-time[k-1])*Volume)
temperature_shifted = [np.average([temperatures[n+1],temperatures[n]]) for n in range(len(temperatures)-1)]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log
  """
